In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [1]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments
)
from datasets import load_dataset

dataset = load_dataset('hakurei/open-instruct-v1', split='train')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/498813 [00:00<?, ? examples/s]

GPT is decoder only model, i.e. it does not take input, but only produce output. So we give it prompt and let it continue from the prompt.

During training, we feed it the prompt and mask the ending tokens, then let the model complete it token by token.

In [ ]:
def preprocess(example):
    """preprocess a row from the dataset
    """
    example['prompt'] = f"{example['instruction']} {example['input']} {example['output']}"
    return example
​
def tokenize_dataset(dataset):
    tokenized_dataset = dataset.map(lambda example: tokenizer(example['prompt'], truncation=True, max_length=128),
                                   batched=True, remove_columns=['prompt'])
    return tokenized_dataset

In [ ]:
# this dataset format is used to train a lot of chatGPT variants
# this is to teach the model to follow the instructions to product output
dataset

In [ ]:
dataset.to_pandas().sample(20)

In [ ]:
dataset = dataset.map(preprocess, remove_columns=['instruction', 'input', 'output'])

In [ ]:
dataset.to_pandas().sample(20)

In [ ]:
dataset = dataset.shuffle(seed=42).select(range(100000)).train_test_split(test_size=0.1)
train_dataset = dataset['train']
test_dataset = dataset['test']
# this is a model half-trained, still bad at answering questions. We will train it to answer better.
MODEL_NAME = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# GPT was trained using sliding windows, so it does not need padding. So no padding token.
# But we need to keep our instruction and context separate, so we need padding and have to define
# the padding token.
tokenizer.pad_token = tokenizer.eos_token

train_dataset = tokenize_dataset(train_dataset)
test_dataset = tokenize_dataset(test_dataset)

In [ ]:
train_dataset, test_dataset

In [ ]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

In [ ]:
# used for masking tokens, batching data, etc.
# we specify a tokenizer for the padding.
# we are using CausalLM not MaskingLM, so mlm=False
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir = './dialoGPT2-instruct',
    num_train_epochs = 1,
    per_device_train_batch_size=8, # normally set it as big as possible without crashing
    per_device_eval_batch_size=16,
)

trainer = Trainer(
    model=model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    data_collator = data_collator
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model()

In [ ]:
model = AutoModelForCausalLM.from_pretrained('./dialoGPT2-instruct').to('cuda')
prompt = ''
def generate_text(prompt):
    inputs = tokenizer.encode(prompt, return_tensors='pt').to('cuda')
    outputs = model.generate(inputs, max_length=64, pad_token_id=tokenizer.eos_token_id)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # remove any words after the last full-stop
    return generated_text[:generated_text.rfind('.')+1]

In [ ]:
generate_text("How to get to Central from Hurstville using public transport in Sydney, Australia?")